# Updating ORM Object

In [1]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import declarative_base

In [2]:
engine = create_engine("sqlite+pysqlite:///example2.db", future=True)
Base = declarative_base(bind=engine)

class Users(Base):
    __table__ = Table("users", Base.metadata, autoload=engine)
    __tablename__ = "users"
    
    def __repr__(self):
        return f'Users(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})'
    
class Email(Base):
    __table__ = Table("email", Base.metadata, autoload=engine)
    __tablename__ = "email"
    
    def __repr__(self):
        return f'Email(email_address={self.email_address!r})'

In [3]:
from sqlalchemy import select
from sqlalchemy.orm import Session

with Session(engine) as session:
    result = session.execute(
        select(Users)
    )
    
    for row in result.all():
        print(row)

(Users(id=1, name='spongebob', fullname='spongebob squarepants'),)
(Users(id=2, name='sandy', fullname='sandy squirrel'),)
(Users(id=3, name='patrick', fullname='patrick star'),)


Selanjutnya data dengan fullname sandy cheeks akan dilakukan update menjadi sandy squirrel.

In [4]:
sandy = session.execute(select(Users).filter_by(name ="sandy")).scalar_one()

In [5]:
sandy

Users(id=2, name='sandy', fullname='sandy squirrel')

In [6]:
sandy in session.dirty

False

In [7]:
sandy.fullname = "sandy squirrel"

Terdapat perubahan pada fullname sandy, dan session mengetahui perubahan ini dan objek tersebut diindikasikan dirty. 

In [8]:
sandy in session.dirty

True

In [9]:
for row in session.execute(select(Users)).all():
    print(row)

(Users(id=1, name='spongebob', fullname='spongebob squarepants'),)
(Users(id=2, name='sandy', fullname='sandy squirrel'),)
(Users(id=3, name='patrick', fullname='patrick star'),)


In [10]:
sandy in session.dirty

False

Pada saat session mengindikasikan suatu objek menjadi dirty, dan apabila terdapat pemanggilan fungsi select maka session akan secara otomatis melakukan flushing sehingga apabila setelah pemanggilan fungsi select dilakukan pengecekan apakah suatu objek dirty maka objek tersebut sudah tidak dirty lagi. Dan untuk melakukan perubahan tinggal memanggil session.commit() saja.

In [11]:
session.commit()

In [12]:
with Session(engine) as session:
    result = session.execute(
        select(Users)
    )
    
    for row in result.all():
        print(row)

(Users(id=1, name='spongebob', fullname='spongebob squarepants'),)
(Users(id=2, name='sandy', fullname='sandy squirrel'),)
(Users(id=3, name='patrick', fullname='patrick star'),)


Sekarang fullname sandy sudah berubah menjadi sandy squirrel. Selain dengan cara terxebut di atas untuk merubah data atau baris tersebut, masih terdapat cara yang lain seperti berikut ini:

In [13]:
from sqlalchemy import update

In [14]:
sandy = session.execute(select(Users).filter_by(name ="sandy")).scalar_one()

In [15]:
sandy

Users(id=2, name='sandy', fullname='sandy squirrel')

In [16]:
sandy.fullname

'sandy squirrel'

In [17]:
session.execute(
    update(Users).where(Users.name == "sandy").
    values(fullname = "sandy squirrel extraordinaire")
)

In [18]:
sandy.fullname

'sandy squirrel extraordinaire'

In [19]:
session.commit()

In [20]:
with Session(engine) as session:
    result = session.execute(
        select(Users)
    )
    
    for row in result.all():
        print(row)

(Users(id=1, name='spongebob', fullname='spongebob squarepants'),)
(Users(id=2, name='sandy', fullname='sandy squirrel extraordinaire'),)
(Users(id=3, name='patrick', fullname='patrick star'),)
